# Recolección de imágenes satelitales

Importación de librerías necesarias

In [7]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import re

Cargar csv 

In [5]:
# Cargamos el archivo con las fechas y horas
ruta_eventos = "../data/raw/evento_granizo_limpio_hora.csv"
df_eventos = pd.read_csv(ruta_eventos, parse_dates=['fecha'])

print("✅ Archivo de eventos con horas cargado exitosamente.")
display(df_eventos.head())

✅ Archivo de eventos con horas cargado exitosamente.


,fecha,ubicacion,fuente,estacion_asignada,hora_aprox
0,2000-01-01,Aeropuerto El Plumerillo (Norte de Mendoza),researchgate.net,"MENDOZA AERO, AR",22:45 - 23:20 UTC
1,2002-03-08,Puente de Hierro (Guaymallén),argentina.gob.ar,"MENDOZA AERO, AR",NaN
2,2005-02-14,Luján de Cuyo,eluniverso.com,"MENDOZA AERO, AR",Tarde
3,2006-02-12,"Luján de Cuyo (Perdriel, Vistalba)",Diario Los Andes,"MENDOZA AERO, AR",NaN
4,2008-01-31,Ciudad de Mendoza,mdzol.com,"MENDOZA AERO, AR",20:20 - 20:37 hs


Verificamos los horarios únicos del dataset

In [6]:
# Obtenemos y mostramos todos los valores únicos de la columna 'hora_aprox'
valores_unicos_hora = df_eventos['hora_aprox'].unique()

print("--- Horarios Únicos Encontrados en tu CSV ---")
print(valores_unicos_hora)

--- Horarios Únicos Encontrados en tu CSV ---
['22:45 - 23:20 UTC' nan 'Tarde' '20:20 - 20:37 hs'
 'Cerca de las 14:00 hs' '18:20 hs' 'Mediodía' 'Tarde (aprox. 18:00 hs)'
 'Tarde (después de las 16:00 hs)' '08:30 hs' 'Noche (21:00 - 01:00 hs)'
 'Noche' 'Tarde (desde las 13:00 hs)' 'Madrugada'
 'Tarde-Noche (15:00 - 19:00 hs)' 'Tarde-Noche' '17:15 hs'
 'Tarde-Noche (aprox. 20:00 hs)' 'Tarde (16:00 hs)' '16:00 - 21:30 hs'
 'Tarde (desde las 16:00 hs)' 'Tarde (desde las 16:45 hs)'
 'Noche (aprox. 23:00 hs)' 'Madrugada (aprox. 01:30 hs)']


In [ ]:
Script para obtener imagenes

1) La función traductora de horas

In [8]:
def parse_hora_a_utc(texto_hora):
    if pd.isna(texto_hora):
        return None # Si no hay dato, devolvemos Nada
    
    texto_hora = str(texto_hora).lower()
    
    # Caso 1: La hora ya está en UTC
    if 'utc' in texto_hora:
        numeros = re.findall(r'\d+', texto_hora)
        if numeros:
            return int(numeros[0]) # Devuelve la primera hora que encuentre
            
    # Caso 2: Horas de texto
    if 'tarde' in texto_hora:
        return 20 # 17:00 hs local -> 20:00 UTC
    if 'noche' in texto_hora:
        return 1 # 22:00 hs local -> 01:00 UTC del día siguiente (simplificado)
    if 'madrugada' in texto_hora:
        return 5 # 02:00 hs local -> 05:00 UTC
    if 'mediodía' in texto_hora or 'mediodia' in texto_hora:
        return 15 # 12:00 hs local -> 15:00 UTC
        
    # Caso 3: Extraer el primer número de la cadena
    numeros = re.findall(r'\d+', texto_hora)
    if numeros:
        hora_local = int(numeros[0])
        # Convertimos a UTC y manejamos el cambio de día
        hora_utc = (hora_local + 3) % 24
        return hora_utc
        
    return None # Si no se pudo interpretar, devolvemos Nada


2) Configuración para la descarga

In [9]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'noaa-goes16'
ruta_guardado = "../data/raw/imagenes_satelitales/"
os.makedirs(ruta_guardado, exist_ok=True)
print("Iniciando la descarga de imágenes satelitales...")

Iniciando la descarga de imágenes satelitales...


3) Bucle principal 

In [10]:
for index, evento in df_eventos.iterrows():
    # Usamos nuestra nueva función para obtener la hora UTC
    hora_utc = parse_hora_a_utc(evento['hora_aprox'])
    
    # Solo procedemos si obtuvimos una hora válida
    if hora_utc is not None:
        try:
            fecha = evento['fecha']
            año = fecha.year
            dia_del_año = fecha.dayofyear
            
            prefix = f"ABI-L2-CMIPF/{año}/{dia_del_año:03d}/{hora_utc:02d}/"
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
            
            if 'Contents' in response:
                key_archivo = response['Contents'][0]['Key']
                nombre_archivo_local = f"{ruta_guardado}{fecha.strftime('%Y-%m-%d')}_{hora_utc:02d}00_UTC.nc"
                
                if not os.path.exists(nombre_archivo_local):
                    print(f"-> Descargando imagen para {fecha.strftime('%Y-%m-%d')} a las {hora_utc:02d}h UTC...")
                    s3.download_file(bucket_name, key_archivo, nombre_archivo_local)
                    print(f"✅ Guardado como: {nombre_archivo_local}")
                else:
                    print(f"-> Archivo ya existe: {nombre_archivo_local}. Saltando.")
            else:
                print(f"-> No se encontraron imágenes para {fecha.strftime('%Y-%m-%d')} a las {hora_utc:02d}:00 UTC.")
        except Exception as e:
            print(f"-> ERROR procesando el evento del {evento['fecha'].strftime('%Y-%m-%d')}: {e}")

print("\n--- Proceso de descarga finalizado ---")

-> No se encontraron imágenes para 2000-01-01 a las 22:00 UTC.
-> No se encontraron imágenes para 2005-02-14 a las 20:00 UTC.
-> No se encontraron imágenes para 2008-01-31 a las 23:00 UTC.
-> No se encontraron imágenes para 2008-11-26 a las 17:00 UTC.
-> No se encontraron imágenes para 2008-12-18 a las 21:00 UTC.
-> No se encontraron imágenes para 2010-09-28 a las 15:00 UTC.
-> No se encontraron imágenes para 2011-02-23 a las 20:00 UTC.
-> No se encontraron imágenes para 2011-11-20 a las 20:00 UTC.
-> No se encontraron imágenes para 2012-10-29 a las 11:00 UTC.
-> No se encontraron imágenes para 2013-02-09 a las 01:00 UTC.
-> No se encontraron imágenes para 2014-10-26 a las 01:00 UTC.
-> No se encontraron imágenes para 2014-12-09 a las 20:00 UTC.
-> No se encontraron imágenes para 2014-12-16 a las 05:00 UTC.
-> No se encontraron imágenes para 2015-09-25 a las 20:00 UTC.
-> No se encontraron imágenes para 2016-03-01 a las 01:00 UTC.
-> Descargando imagen para 2017-04-05 a las 20h UTC...
